# Assignment 1: Market Clearing (System Perspective)

### Import external libraries

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np 
from network import initialize_network, system_demand

## Step 0: Build a Relevant Case Study

Please select an electric power network from the following options:
1. IEEE 24-bus reliability test system: link.
2. IEEE reliability test system (2019 update): link.
3. IEEE power systems test cases (various cases with 14, 30, 57, 118, and 300 buses): link.
4. Case studies available in the open-source Julia platform PowerModels.jl: link.

You are also free to choose another case study. If some data is missing, please select reason-
able arbitrary values. For technical details on conventional generators and transmission lines, this link may be helpful (it corresponds to the IEEE 24-bus case study, but similar data can be used for other cases).

### IEEE 24-bus reliability test system

In [77]:
consumers, generators = initialize_network()
print(generators)
print(consumers)
print("Network initialized successfully.")

[Generator(unit=1, node=1), Generator(unit=2, node=2), Generator(unit=3, node=7), Generator(unit=4, node=13), Generator(unit=5, node=15), Generator(unit=6, node=15), Generator(unit=7, node=16), Generator(unit=8, node=18), Generator(unit=9, node=21), Generator(unit=10, node=22), Generator(unit=11, node=23), Generator(unit=12, node=23)]
[Consumer(load=1, node=1, share=0.038), Consumer(load=2, node=2, share=0.034), Consumer(load=3, node=3, share=0.063), Consumer(load=4, node=4, share=0.026), Consumer(load=5, node=5, share=0.025), Consumer(load=6, node=6, share=0.048), Consumer(load=7, node=7, share=0.044), Consumer(load=8, node=8, share=0.06), Consumer(load=9, node=9, share=0.061), Consumer(load=10, node=10, share=0.068), Consumer(load=11, node=13, share=0.093), Consumer(load=12, node=14, share=0.068), Consumer(load=13, node=15, share=0.111), Consumer(load=14, node=16, share=0.035), Consumer(load=15, node=18, share=0.117), Consumer(load=16, node=19, share=0.064), Consumer(load=17, node=20

### Additional Assumptions


*   Assume that the price bids of all producers are non-negative and equal to their marginal production cost. In particular, the production cost of renewable units is assumed to be zero. Additionally, these units offer their forecasted capacity, meaning their offer quantities vary over time.

*   For the bid price of price-elastic demands, use comparatively high values (relative to the generation cost of conventional units) to ensure that most demands are supplied. For inspiration, check the real bid price data in Nord Pool [link].
*   A potential source for wind power forecast data is available at this link (you may nor- malize the data to fit your case study). Another potential source for the renewable power generation data is renewables.ninja.
*   For transmission lines, you may assume a uniform reactance for all lines (e.g., 0.002 p.u., leading to a susceptance of 500 p.u.).

## Step 1: Copper-Plate, Single Hour

In Lecture 2, you learn how to develop a market-clearing optimization model for a copper-plate
power system (i.e., without modeling the transmission network) in a single-hour setting.
Please determine the following market-clearing outcomes:


### Qur Optimzation Problem


\begin{align}
\textrm{minimize} \quad
&13.32x_1 + 13.32x_2 + 20.7x_3 + 20.93x_4 + 26.11x_5 + 10.52x_6 \\
&\quad + 10.52x_7 + 6.02x_8 + 5.47x_9 + 0x_{10} + 10.52x_{11} + 10.89x_{12} \\
\textrm{subject to} \\
&0 \le  x_1 \le 152 \\
&0 \le x_1 \le 152 \\
&0 \le x_2 \le 152 \\
&0 \le x_3 \le 350 \\
&0 \le x_4 \le 591 \\
&0 \le x_5 \le 60 \\
&0 \le x_6 \le 155 \\
&0 \le x_7 \le 155 \\
&0 \le x_8 \le 400 \\
&0 \le x_9 \le 400 \\
&0 \le x_{10} \le 300 \\
&0 \le x_{11} \le 310 \\
&0 \le x_{12} \le 350 \\
& 2464.965 - \sum_{i=1}^{12} x_i = 0 \\
\end{align}


\begin{align}
\textrm{minimize} \quad 
&\sum_{i=1}^{12} C_i *x_i \\
\end{align}

\begin{align}
\textrm{where} \quad 
C_i = \textrm{offer price of generator i} \\
x_i =  \textrm{production of generator i} \\
\end{align}



### The market-clearing price under a uniform pricing scheme.

In [82]:
# define the demand time at 16:00
total_consumption_16 = system_demand[16]
print(f"Total consumption at 16:00 is {total_consumption_16} MW")
#check if all generators are there
id_Gnerators = []
for i in generators:
    id_Gnerators.append(i.unit_id)
print(id_Gnerators)

Total consumption at 16:00 is 2464.965 MW
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


Define the supply


In [83]:
#Variable generators costs 
generator_cost = []
for i in generators:
    generator_cost.append(i.cost_energy)
print(generator_cost)

[13.32, 13.32, 20.7, 20.93, 26.11, 10.52, 10.52, 6.02, 5.47, 0, 10.52, 10.89]


Define Generator capacity:

In [84]:
#Generators capacity
generator_capacity = []
for i in generators:
    generator_capacity.append(i.p_max)
print(generator_capacity)

[152, 152, 350, 591, 60, 155, 155, 400, 400, 300, 310, 350]


Everytime you use Gurobi, you will need to import the package ```gurobipy```. The specific module ```GRB``` is commonly imported separately, as it is used frequently. 

In [85]:
#create the optimization model
model = gp.Model("Economic_Dispatch_Model")

In [86]:
#create decision variables
production_variables = [model.addVar(lb=0, ub = float('inf'), vtype=GRB.CONTINUOUS, name=f"p_{i.unit_id}") for i in generators]

In [87]:
#add constraint
balance_constraint = model.addLConstr(gp.quicksum(production_variables[i] for i in range(len(generators))) == total_consumption_16, name="balance")
capacity_constraints = [model.addLConstr(production_variables[i], GRB.LESS_EQUAL, generator_capacity[i], name=f"capacity_{i}") for i in range(len(generators))]


In [88]:
# set objective function
model.setObjective(gp.quicksum(generator_cost[i] * production_variables[i] for i in range(len(generators))), GRB.MINIMIZE)


In [89]:
model.optimize()

Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (mac64[rosetta2] - Darwin 25.2.0 25C56)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13 rows, 12 columns and 24 nonzeros (Min)
Model fingerprint: 0xf455b892
Model has 11 linear objective coefficients
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+03]

Presolve removed 12 rows and 3 columns
Presolve time: 0.01s
Presolved: 1 rows, 9 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.706206e+02   0.000000e+00      0s
       1    2.0862156e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.086215550e+04


In [90]:
#check status and print results 
if model.status == GRB.OPTIMAL:
    optimal_objective = model.objVal
    optimal_production_variables = [production_variables[i].x for i in range(len(generators))]
    balance_dual = balance_constraint.Pi
    capacity_duals = [capacity_constraints[i].Pi for i in range(len(generators))]

print (f"Optimal objective value: {optimal_objective}")
for index, optimal in enumerate(optimal_production_variables):
    print(f"Optimal production for Generator {id_Gnerators[index]}: {optimal} MW")

print(f"Dual variable for balance constraint: {balance_dual}")
for index, dual in enumerate(capacity_duals):
    print(f"Dual variable for capacity constraint of Generator {id_Gnerators[index]}: {dual}")

else:
    print(f"optimization of model {model.ModelName} was not successful. Status code: {model.status}")

Optimal objective value: 20862.155500000004
Optimal production for Generator 1: 152.0 MW
Optimal production for Generator 2: 152.0 MW
Optimal production for Generator 3: 90.96500000000015 MW
Optimal production for Generator 4: 0.0 MW
Optimal production for Generator 5: 0.0 MW
Optimal production for Generator 6: 155.0 MW
Optimal production for Generator 7: 155.0 MW
Optimal production for Generator 8: 400.0 MW
Optimal production for Generator 9: 400.0 MW
Optimal production for Generator 10: 300.0 MW
Optimal production for Generator 11: 310.0 MW
Optimal production for Generator 12: 350.0 MW
Dual variable for balance constraint: 20.7
Dual variable for capacity constraint of Generator 1: -7.379999999999999
Dual variable for capacity constraint of Generator 2: -7.379999999999999
Dual variable for capacity constraint of Generator 3: 0.0
Dual variable for capacity constraint of Generator 4: 0.0
Dual variable for capacity constraint of Generator 5: 0.0
Dual variable for capacity constraint of G

In [91]:
print(f"The markets clearing price at 16:00 is: {balance_dual} $/MWh")

The markets clearing price at 16:00 is: 20.7 $/MWh


### The total operating cost and social welfare of the system.

In [92]:
print(f"Total cost of generation at 16:00 is: {optimal_objective} $")
print(f"Social welfare at 16:00 is: {total_consumption_16 * balance_dual - optimal_objective} $")

Total cost of generation at 16:00 is: 20862.155500000004 $
Social welfare at 16:00 is: 30162.62 $


### The profit of each producer, including both conventional units and wind farms.

In [94]:
print(f"The production cost for each generator at 16:00 is:")
for i in range(len(generators)):
    cost = generator_cost[i] * optimal_production_variables[i]
    print(f"Generator {id_Gnerators[i]}: {cost} $")

The production cost for each generator at 16:00 is:
Generator 1: 2024.64 $
Generator 2: 2024.64 $
Generator 3: 1882.975500000003 $
Generator 4: 0.0 $
Generator 5: 0.0 $
Generator 6: 1630.6 $
Generator 7: 1630.6 $
Generator 8: 2408.0 $
Generator 9: 2188.0 $
Generator 10: 0.0 $
Generator 11: 3261.2 $
Generator 12: 3811.5 $


### The utility of each demand, defined as:
$$

    \begin{align}
        \textrm{Utility = Power Consumption ×(Bid Price−Market-Clearing Price)}
    \end{align}

$$

In [ ]:
for consumer in consumers:
    consuption = consumer.share * total_consumption_16
    bid_price = ??
    utility = consuption * (bid_price - balance_dual)
    print(f"Consumer {consumer.consumer_id}: Consumption = {consuption} MW, Utility = {utility} $") 

SyntaxError: invalid syntax (603326317.py, line 4)